## 비모수 이표본 가설검정

- 모수적 방법 : 가정 O, t-test - 평균 비교
- 비모수적 방법 : 가정 X, 윌콕슨 - 중앙값 비교

### 11.3.3 윌콕슨 부호순위 검정

#### 근력운동 후 집중력이 향상했는지?

1) 두 표본의 차이를 구함

2) 절댓값이 작은 데이터 부터 순서대로 나열 --> 순위 나열

3) 부호가 마이너스인 것(r_minus)끼리, 부호가 플러스인 것(r_plus)끼리 순위 합함

4) 둘 중, 작은 것을 검정통계량으로 선택

#####  python :  scipy.stats의 wilcoxon 사용

In [4]:
#데이터 형태

import pandas as pd
import numpy as np
from scipy import stats

training_rel = pd.read_csv("..\kong_Dacon contest\hanbit_data\ch11_training_rel.csv")
toy_df = training_rel[:6].copy()
toy_df.head()

,전,후
0,59,41
1,52,63
2,55,68
3,61,59
4,59,84


In [5]:
# 1) 두 표본의 차이를 구함
diff = toy_df['후'] - toy_df['전']
toy_df['차'] = diff
toy_df

,전,후,차
0,59,41,-18
1,52,63,11
2,55,68,13
3,61,59,-2
4,59,84,25
5,45,37,-8


In [6]:
# 2) 절댓값이 작은 데이터 부터 순서대로 나열 --> 순위 나열
rank = stats.rankdata(abs(diff)).astype(int)
toy_df['순위'] = rank
toy_df

,전,후,차,순위
0,59,41,-18,5
1,52,63,11,3
2,55,68,13,4
3,61,59,-2,1
4,59,84,25,6
5,45,37,-8,2


In [7]:
#3) 부호가 마이너스인 것(r_minus)끼리, 부호가 플러스인 것(r_plus)끼리 합함
r_minus = np.sum((diff < 0) * rank)
r_plus = np.sum((diff > 0) * rank)

r_minus, r_plus

# 검정통계량은 8이 됨 > 단측검정 > 임계값 보다 작으면 기각됨 

(8, 13)

In [8]:
### python
T, p = stats.wilcoxon(training_rel['후'], training_rel['전'])
p

0.039989471435546875

In [9]:
# 차이의 데이터를 넣어도 됨
T, p = stats.wilcoxon(training_rel['후'] - training_rel['전'])
p

0.039989471435546875

##### 순위 합이 작은 집단의 값을 편향으로 사용하는 이유

* 차이에 편향이 생길수록, r_minus, r_plus에도 편향이 생기고 > 검정통계량이 작아짐

- 극단적인 예시 : 전부 집중력 향상 vs 집중력 향상 반 + 감소 반 

In [10]:
###### 전부 집중력 향상된 경우
# 순위 1,2,3,4,5,6를 가진 데이터 만들기
# (참고로 동점자는 제거 혹은 평균으로 다루기 때문에 차가 1,2,3,4,5,6 나는 데이터로 사용 )

toy_df['후'] = toy_df['전'] + np.arange(1, 7)
diff = toy_df['후'] - toy_df['전']
rank = stats.rankdata(abs(diff)).astype(int)
toy_df['차'] = diff
toy_df['순위'] = rank
toy_df

,전,후,차,순위
0,59,60,1,1
1,52,54,2,2
2,55,58,3,3
3,61,65,4,4
4,59,64,5,5
5,45,51,6,6


In [11]:
# 마이너스끼리, 플러스끼리 합침

r_minus = np.sum((diff < 0) * rank)
r_plus = np.sum((diff > 0) * rank)

r_minus, r_plus

# 검정통계량은 0이됨 (=전부 집중력 향상한 경우. 효과가 있을 경우 검정통계량이 작아진다.)

(0, 21)

In [12]:
###### 집중력 향상 된 사람 반, 떨어진 사람 반 있을 경우

toy_df['후'] = toy_df['전'] + [1, -2, -3, 4, 5, -6]
diff = toy_df['후'] - toy_df['전']
rank = stats.rankdata(abs(diff)).astype(int)
toy_df['차'] = diff
toy_df['순위'] = rank
toy_df

,전,후,차,순위
0,59,60,1,1
1,52,50,-2,2
2,55,52,-3,3
3,61,65,4,4
4,59,64,5,5
5,45,39,-6,6


In [13]:
r_minus = np.sum((diff < 0) * rank)
r_plus = np.sum((diff > 0) * rank)

r_minus, r_plus

#이때 검정통계량은 10이 됨. > (=집중력 향상 효과가 반반인 경우. 즉, 효과가 없을 경우 검정통계량이 커진다.)

(11, 10)

# 11.3.4 만.위트니 U검정  

1) 모든 데이터에 값이 작은 순서대로 순위를 부여

2) 한 그룹의 순위합 사용 (여기선 A그룹)

3) 검정통계량 U를 구해 가설검정 : 검정통계량 U = 순위합에 n(n+1)/2를 빼줌

4) 양측검정 시행

##### Python : spicy.stats의 mannwhitneyu 사용

In [14]:
#데이터 만들기
training_ind = pd.read_csv('..\kong_Dacon contest\hanbit_data\ch11_training_ind.csv')
toy_df = training_ind[:5].copy()
toy_df

,A,B
0,47,49
1,50,52
2,37,54
3,60,48
4,39,51


In [15]:
# 1) 모든 데이터에 값이 작은 순서대로 순위를 부여

rank = stats.rankdata(np.concatenate([toy_df['A'],
                                      toy_df['B']]))
rank_df = pd.DataFrame({'A': rank[:5],
                        'B': rank[5:10]}).astype(int)
rank_df

,A,B
0,3,5
1,6,8
2,1,9
3,10,4
4,2,7


In [16]:
#2) 한 그룹의 순위합 사용 (여기선 A그룹)
#3) 검정통계량 U를 구해 가설검정 : 검정통계량 U = 순위합에 n(n+1)/2를 빼줌.

n1 = len(rank_df['A'])
u = rank_df['A'].sum() - (n1*(n1+1))/2
u

7.0

In [17]:
## 파이썬으로 검정시
u, p = stats.mannwhitneyu(training_ind['A'], training_ind['B'],
                          alternative='two-sided')
p

0.05948611166127324

어떤 그룹의 합을 사용해도 데이터 편향을 잘 반영해줌. > 양측검정 시행

- 데이터에 좋은 순위가 모여있거나 vs 나쁜 순위가 모여있는 경우


In [18]:
# 데이터에 좋은 순위가 모여있는 경우
# 데이터 만들기
rank_df = pd.DataFrame(np.arange(1, 11).reshape(2, 5).T,
                       columns=['A', 'B'])
rank_df

,A,B
0,1,6
1,2,7
2,3,8
3,4,9
4,5,10


In [19]:
#검정통계량 구하기
u = rank_df['A'].sum() - (n1*(n1+1))/2
u


0.0

In [20]:
##나쁜 순위가 모여있는 경우
#데이터 만들기

rank_df = pd.DataFrame(np.arange(1, 11).reshape(2, 5)[::-1].T,
                       columns=['A', 'B'])
rank_df

,A,B
0,6,1
1,7,2
2,8,3
3,9,4
4,10,5


In [21]:
#검정통계량 구하기
u = rank_df['A'].sum() - (n1*(n1+1))/2
u


25.0

# 11.3.5 카이제곱 검정

##### A광고 B광고 동시에 내보냈을 때, 효과가 있는가? (=구입 비율에 유의미한 차이가 있는가?)
- 광고 종류와 상품 구입 유무가 독립이 아니면, 유의미한 차이가 나올 것
- 광고 종류와 상품 구입 유무가 독립이면, 유의미한 차이 없을 것임

1) 교차집계표 만들기

2) 각 셀의 기대도수를 구함 : 두 변수가 독립일 때 기대되는 도수

3) 각 관측치에서 구한 기대도수를 빼서 검정통계량을 구함

4) 임계값이나 p값과 비교해서 가설검정

##### python : scipy.stats의 chi2_contingency 사용

In [51]:
# 데이터 불러오기

ad_df = pd.read_csv('..\kong_Dacon contest\hanbit_data\ch11_ad.csv')
n = len(ad_df)
print(n)
ad_df.head()

1000


,광고,구입
0,B,하지 않았다
1,B,하지 않았다
2,A,했다
3,A,했다
4,B,하지 않았다


In [53]:
# python : scipy.stats의 chi2_contingency 사용¶
chi2, p, dof, ef = stats.chi2_contingency(ad_cross,
                                          correction=False)
chi2, p, dof # 검정통계량, # 자유도, #기대도수


(3.75, 0.052807511416113395, 1)

In [54]:
ef #기대도수

array([[360.,  40.],
       [540.,  60.]])

###### 교차집계표 (cross table, 분할표)

- python pandas의 crosstab 함수로 작성 가능


In [39]:
# 1) 교차집계표 만들기

ad_cross = pd.crosstab(ad_df['광고'], ad_df['구입'])
ad_cross

구입,하지 않았다,했다
광고,,
A,351,49
B,549,51


In [40]:
# 광고를 통해 상품 구입한 비율

ad_cross['했다'] / (ad_cross['했다'] + ad_cross['하지 않았다'])

광고
A    0.1225
B    0.0850
dtype: float64

In [43]:
# 2-1) 각 셀의 기대도수를 구함 : 두 변수가 독립일 때 기대되는 도수

# 구입하지 않은 사람의 합계 / 상품을 구입한 사람의 합계
n_not, n_yes = ad_cross.sum()
n_not, n_yes

# 구입한 사람의 합계의 비율 10%

(900, 100)

In [44]:
# 2-2) 각 셀의 기대도수를 구함 : 두 변수가 독립일 때 기대되는 도수

# A를 본 사람의 합계, #B를 본 사람의 합계
n_adA, n_adB = ad_cross.sum(axis=1)
n_adA, n_adB

(400, 600)

여기서 광고의 구입이 독립이어서 광고에 따라 상품을 구입한 비율이 변하지 않는다면,


            A를 보고 상품을 구입하는 경우, 400명 중 10%인 40명이 상품을 구입할 수 있을 것으로 기대 (기대도수)
            
            B는 60명이 상품을 구입할 수 있을 것으로 기대 (기대도수)
            

In [45]:
# 3-1) 각 관측치에서 구한 기대도수를 빼서

ad_ef = pd.DataFrame({'했다': [n_adA * n_yes / n,
                              n_adB * n_yes / n],
                      '하지 않았다': [n_adA * n_not / n,
                                   n_adB * n_not / n]},
                      index=['A', 'B'])
ad_ef

,했다,하지 않았다
A,40.0,360.0
B,60.0,540.0


In [46]:
# 3-2) 검정통계량을 구함

y = ((ad_cross - ad_ef) ** 2 / ad_ef).sum().sum()
y

3.75

In [49]:
# 4) 임계값이나 p값과 비교해서 가설검정
rv = stats.chi2(1)
1 - rv.cdf(y)


0.052807511416113395